In [1]:
import requests
import pandas as pd
import scipy.stats as stat
from datetime import date
from config import gkey
from citipy import citipy

import json
import gmaps

In [2]:
dfCities = pd.read_csv('Cities.csv')

In [12]:
gmaps.configure(api_key = gkey)

location = dfCities[['Latitude', 'Longitude']]
humidity = dfCities['Humidity %']

fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(location, weights=humidity, max_intensity = dfCities['Humidity %'].max())

fig.add_layer(heatmap)

In [13]:
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
dfGoodCities = dfCities.loc[(dfCities['Temperature (F)'] >=70) & (dfCities['Temperature (F)'] < 80) & 
                            (dfCities['Wind Speed (mph)'] < 10) &
                            (dfCities['Cloudiness %'] < 5)]
dfGoodCities

,City,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph),Latitude,Longitude,Country
72,esmeralda,77.67,68,0,3.65,21.85,-78.12,CU
167,altar,73.99,25,0,4.00,30.70,-111.82,MX
172,carnarvon,78.85,63,1,8.41,-24.87,113.63,AU
279,khed,73.45,67,0,3.49,17.72,73.38,IN
284,teknaf,73.20,88,1,2.17,20.86,92.31,BD
292,colombo,77.20,84,0,3.98,6.93,79.85,LK
296,loandjili,79.45,85,1,5.06,-4.76,11.86,CG
316,san juan,70.68,32,0,5.95,-31.54,-68.54,AR
419,chabahar,78.04,62,0,1.48,25.29,60.64,IR


In [6]:
params = {'type': 'lodging',
          'keyword': 'hotel',
          'radius': 5000,
          'key': gkey,
          'language': 'en'}

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

pins = []
for index, row in dfGoodCities.iterrows():
    params['location'] = f'{row["Latitude"]},{row["Longitude"]}'
    
    hotelJson = requests.get(url, params = params).json()
    try:
        latlon = (hotelJson['results'][0]['geometry']['location']['lat'], hotelJson['results'][0]['geometry']['location']['lng'] )

        city = row['City']
        name = hotelJson['results'][0]['name']
        country = row['Country']

        pins.append({'latlon': latlon,
                 'city': city,
                 'name': name,
                 'country': country})
    except:
        print(f'No Hotel found for {row["City"]}')
    

In [7]:
locations = [pin['latlon'] for pin in pins]

#stole this bit from https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#drawing-markers-lines-and-polygons
infoTemplate = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
#------------

markerInfo = [infoTemplate.format(**pin) for pin in pins]

markerLayer = gmaps.marker_layer(locations, info_box_content=markerInfo)
fig.add_layer(markerLayer)
fig

Figure(layout=FigureLayout(height='420px'))